# The Below Notebook is for a text generation model with LSTM.

Wish me luck.


### The alternative is to bring in movies that are mainly women and use those scripts:

- Bridesmaids
- Charlie's Angels
- Ghost World
- Juno
- Martha Marcy May Marlene
- Precious
- Sex and the City
- The Miracle Worker
- The Help
- Frozen

To be honest I think that, at around 20,000 words apiece, we are looking at a big enough corpus to just about get something going. Well, I guess the only way to find out is to try.


Many thanks to Jason Brownlee, whose [article](https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/) provided much of the basis for what you see below.

In [124]:
# Bring some mates

import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout, Embedding
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import sys
import re

In [5]:
#from google.colab import drive
#drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### We must create a dataset of the screenplays that we want to use for training.

The screenplays that we are going to use is above.

In [102]:
#Lets save each script to be used as its own variable here
bridesmaids = "../data/Scripts/BRIDESMAIDS.TXT"
charlies_angels = "../data/Scripts/CHARLIE'S ANGELS.TXT"
ghost_world = "../data/Scripts/GHOST WORLD.TXT"
juno = "../data/Scripts/JUNO.TXT"
martha_marcy = "../data/Scripts/MARTHA MARCY MAY MARLENE.TXT"
precious = "../data/Scripts/PRECIOUS.TXT"
sex_city = "../data/Scripts/SEX AND THE CITY: THE MOVIE.TXT"
miracle_worker = "../data/Scripts/THE MIRACLE WORKER.TXT"
the_help = "../data/Scripts/THE HELP.TXT"
frozen = "../data/Scripts/FROZEN.TXT"

In [112]:
all_scripts = open(bridesmaids, 'r', encoding='utf-8').read() + open(charlies_angels, 'r', encoding='utf-8').read() + open(ghost_world, 'r', encoding='utf-8').read() + open(juno, 'r', encoding='utf-8').read() + open(martha_marcy, 'r', encoding='utf-8').read() + open(precious, 'r', encoding='utf-8').read() + open(sex_city, 'r', encoding='utf-8').read() + open(miracle_worker, 'r', encoding='utf-8').read() + open(the_help, 'r', encoding='utf-8').read() + open(frozen, 'r', encoding='utf-8').read()
raw_text = all_scripts
raw_text = raw_text.lower()

In [73]:
raw_text

'                                 bridesmaids\n\n                                 written by\n\n                                annie mumolo\n\n                                                           8/12/09\n\n          int. clean, upscale. modern bathroom.\n\n          annie is in nice lingerie, happily getting ready for the\n\n          day. she puts a lot of effort into her appearance.\n\n          products everywhere, too many hair-flips, eye-cream, no\n\n          hair out of place.\n\n          she walks into the bedroom, where ted, a gorgeous man\n\n          sleeps. she sneaks back into the bed, carefully\n\n          positioning herself and moving the sheets to show her\n\n          good parts.\n\n          she coughs and nudges ted to wake him up then pretend\n\n          sleeps.\n\n                         ted\n\n          good morning.\n\n          annie pretends to wake up.\n\n                         ted (cont\'d)\n\n          wow, you look beautiful.\n\n             

In [113]:
tokens = re.findall(r"\w+", raw_text)

In [115]:
tokens = [word for word in tokens if not word.startswith('0')]

In [116]:
tokens

['bridesmaids',
 'written',
 'by',
 'annie',
 'mumolo',
 '8',
 '12',
 'int',
 'clean',
 'upscale',
 'modern',
 'bathroom',
 'annie',
 'is',
 'in',
 'nice',
 'lingerie',
 'happily',
 'getting',
 'ready',
 'for',
 'the',
 'day',
 'she',
 'puts',
 'a',
 'lot',
 'of',
 'effort',
 'into',
 'her',
 'appearance',
 'products',
 'everywhere',
 'too',
 'many',
 'hair',
 'flips',
 'eye',
 'cream',
 'no',
 'hair',
 'out',
 'of',
 'place',
 'she',
 'walks',
 'into',
 'the',
 'bedroom',
 'where',
 'ted',
 'a',
 'gorgeous',
 'man',
 'sleeps',
 'she',
 'sneaks',
 'back',
 'into',
 'the',
 'bed',
 'carefully',
 'positioning',
 'herself',
 'and',
 'moving',
 'the',
 'sheets',
 'to',
 'show',
 'her',
 'good',
 'parts',
 'she',
 'coughs',
 'and',
 'nudges',
 'ted',
 'to',
 'wake',
 'him',
 'up',
 'then',
 'pretend',
 'sleeps',
 'ted',
 'good',
 'morning',
 'annie',
 'pretends',
 'to',
 'wake',
 'up',
 'ted',
 'cont',
 'd',
 'wow',
 'you',
 'look',
 'beautiful',
 'annie',
 'acting',
 'embarrassed',
 'what'

In [117]:
# create mapping of unique words to integers
# Change this to raw_text if you just want characters
chars = sorted(list(set(tokens)))

In [67]:
#remove_chars = ['<', '>', '~', '^', '{', '}', '@', '%', '+', '=', '[', ']', '_', '#']
#chars = [i for i in chars if i not in remove_chars]

In [118]:
# create mapping of unique words to integers
# Change this to raw_text if you just want characters
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [119]:
print(chars)

['1', '10', '100', '101', '101a', '102', '103', '104', '104a', '104b', '104c', '105', '105a', '105b', '105c', '105d', '105e', '106', '107', '107a', '108', '108a', '109', '11', '110', '111', '112', '113', '114', '115', '116', '116a', '117', '118', '118a', '119', '11al3', '12', '120', '120a', '121', '122', '122a', '123', '124', '125', '125a', '125th', '126', '126th', '127', '127a', '128', '129', '13', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '14', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '149a', '15', '150', '151', '152', '153', '154', '155', '156', '157', '158', '158a', '158b', '159', '159a', '159aa', '159ab', '159ac', '159b', '159c', '159d', '159e', '159f', '15a', '15th', '16', '160', '164', '165', '166', '166a', '166b', '168', '168a', '168b', '168c', '168d', '168e', '168l', '16a', '16th', '17', '175', '176', '178', '178a', '178b', '178c', '17th', '18', '180', '183', '184', '185', '186', '187', '188', '1883', '189', '18a', '19',

Use the below for word tokenization. Above is for characters

In [121]:
n_chars = len(tokens)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Unique Words: ", n_vocab)

Total Characters:  216916
Unique Words:  12277


In [122]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = tokens[i:i + seq_length]
	seq_out = tokens[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

Total Patterns:  216816


In [123]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [131]:
# define the LSTM model
model = Sequential()
model.add(Embedding(12278, 100, input_length = seq_length))
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))

# Extra layer, because computers need to work
model.add(LSTM(256))
model.add(Dropout(0.2))

model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [132]:
# define the checkpoint - keep the best one
filepath="word-textgen-weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

### Fit and run and pray!

Everybody hold on to your butts!

In [134]:
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
1694/1694 [==============================] - 1282s 757ms/step - loss: 6.9550

Epoch 00001: loss improved from inf to 6.95502, saving model to word-textgen-weights-improvement-01-6.9550.hdf5
Epoch 2/20
1694/1694 [==============================] - 1236s 730ms/step - loss: 6.8635

Epoch 00002: loss improved from 6.95502 to 6.86348, saving model to word-textgen-weights-improvement-02-6.8635.hdf5
Epoch 3/20
1694/1694 [==============================] - 1229s 726ms/step - loss: 6.8541

Epoch 00003: loss improved from 6.86348 to 6.85409, saving model to word-textgen-weights-improvement-03-6.8541.hdf5
Epoch 4/20
1694/1694 [==============================] - 1227s 724ms/step - loss: 6.8493

Epoch 00004: loss improved from 6.85409 to 6.84929, saving model to word-textgen-weights-improvement-04-6.8493.hdf5
Epoch 5/20
1694/1694 [==============================] - 1230s 726ms/step - loss: 6.8456

Epoch 00005: loss improved from 6.84929 to 6.84565, saving model to word-textgen-weights-improv

In [135]:
# load the network weights
best_model = "../code/word-textgen-weights-improvement-20-6.8268.hdf5"
model.load_weights(best_model)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [136]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [1]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("FADE IN:")
# include a space in the join statement if doing word2Vec
print ("\"", ' '.join([int_to_char[value] for value in pattern]), "\"")

# generate characters
for i in range(50):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nFADE OUT.")

NameError: name 'numpy' is not defined

### Initial Evaluations

The first running of the text generation model involved some hilarious outcomes, namely the fact that, after a while, the only word being produced was, "to". I can't say it's a roaring success, but there's still time and I am going to see what else I can do to fix things from here.

The word2Vec model began spewing the word "the" repeatedly at the end, which is not useful. It seems to only work for around 50 words before it becomes unintelligible. And it also doesn't keep the general structure of a screenplay, which is an issue in itself.